# 08. 윤곽선
- 윤곽선(contour) : 같은 색상(빛의 강도)을 가진 영역의 경계선을 연결한 곡선

## 윤곽선 그리는 방법
1. 엣지를 찾는다(Threshold/Canny)
2. 윤곽선을 생성해서 데이터로 저장(findContours)
3. 이미지 위에 윤곽선을 그림(drawContours)

## 윤곽선 찾기 모드
- `cv2.RETR_EXTERNAL` : 가장 바깥쪽 윤곽선만 가져옴
- `cv2.RETR_LIST` : 모든 윤곽선을 가져오지만 계층 구조 정보는 무시함
- `cv2.RETR_TREE` : 모든 윤곽선을 가져오며, 계층 구조도 포함

## 윤곽선 근사화 방법
- 윤곽선의 점을 얼마나 세밀하게 저장할지 결정
- `cv2.CHAIN_APPROX_NONE` : 윤곽선의 모든 점을 저장, 메모리를 많이 사용함
- `cv2.CHAIN_APPROX_SIMPLE` : 불필요한 중복 점을 제거하여 메모리 사용을 최적화

In [2]:
import cv2 as cv
DOG_PATH = "../images/dog.jpg"

## 8-1. 윤곽선 검출

In [ ]:
img = cv.imread(DOG_PATH)
coppied = img.copy()

# 그레이 스케일로 변환
gray = cv.cvtColor(coppied, cv.COLOR_BGR2GRAY)

# 이진화
ret, binary = cv.threshold(gray, -1, 255, cv.THRESH_BINARY | cv.THRESH_OTSU)

# 윤곽선 찾기
contours, hierachy = cv.findContours(binary, cv.RETR_EXTERNAL, cv.CHAIN_APPROX_NONE)

# 윤곽선 그리기
cv.drawContours(img, contours, -1, (0,255,0), 2)

cv.imshow("contours", img)

cv.waitKey(0)
cv.destroyAllWindows()


In [ ]:
img = cv.imread(DOG_PATH)
coppied = img.copy()

# 그레이 스케일로 변환
gray = cv.cvtColor(coppied, cv.COLOR_BGR2GRAY)

# 이진화
ret, binary = cv.threshold(gray, -1, 255, cv.THRESH_BINARY | cv.THRESH_OTSU)

# Canny사용
canny = cv.Canny(binary, 50, 150)

# 윤곽선 찾기
contours, hierachy = cv.findContours(canny, cv.RETR_LIST, cv.CHAIN_APPROX_NONE)

# 윤곽선 그리기
cv.drawContours(img, contours, -1, (0,255,0), 2)

cv.imshow("contours", img)

cv.waitKey(0)
cv.destroyAllWindows()


## 8-2. boundingRect
- 윤곽선을 둘러싼 사각형

In [15]:
img = cv.imread(DOG_PATH)
coppied = img.copy()

gray = cv.cvtColor(img, cv.COLOR_BGR2GRAY)
ret, binary = cv.threshold(gray, -1, 255, cv.THRESH_BINARY | cv.THRESH_OTSU)
contours, hierachy = cv.findContours(binary, cv.RETR_LIST, cv.CHAIN_APPROX_NONE)

for contour in contours:
    x, y, width, height = cv.boundingRect(contour)
    cv.rectangle(img, (x,y), (x + width, y + height), (255, 255, 0), 2, cv.LINE_AA)


cv.imshow("Bounding Rect", img)
cv.waitKey(0)
cv.destroyAllWindows()

## 8-3. contourArea
- contour의 면적 계산

In [17]:
img = cv.imread(DOG_PATH)
coppied = img.copy()

gray = cv.cvtColor(img, cv.COLOR_BGR2GRAY)
ret, binary = cv.threshold(gray, -1, 255, cv.THRESH_BINARY | cv.THRESH_OTSU)
contours, hierachy = cv.findContours(binary, cv.RETR_LIST, cv.CHAIN_APPROX_NONE)

for contour in contours:
    # contour의 면적이 1000보다 크면
    if cv.contourArea(contour) > 1000:
        x, y, width, height = cv.boundingRect(contour)
        cv.rectangle(img, (x,y), (x + width, y + height), (255, 255, 0), 2, cv.LINE_AA)


cv.imshow("Bounding Rect", img)
cv.waitKey(0)
cv.destroyAllWindows()

In [3]:
# 실습4. 순서대로 박스 표시
VEHICLE_PATH = "../images/vehicles.png"
img = cv.imread(VEHICLE_PATH)
coppied = img.copy()
name = "Vehicles"
cv.namedWindow(name)

gray = cv.cvtColor(img, cv.COLOR_BGR2GRAY)
ret, binary = cv.threshold(gray, -1, 255, cv.THRESH_BINARY | cv.THRESH_OTSU)
contours, hierarchy = cv.findContours(binary, cv.RETR_EXTERNAL, cv.CHAIN_APPROX_SIMPLE)

filtered_contours = [] # 윤곽선 중에서 조건을 만족하는 것만 따로 모아 관리
for contour in contours:
    if cv.contourArea(contour) > 700: # 700px 이하 작은 객체는 제외
        filtered_contours.append(contour) # 조건에 맞는 부분만 filtered_contours에 추가

# Index 트랙바를 움직이면 특정 차량의 인덱스 선택 가능
cv.createTrackbar("Index", name, 0, len(filtered_contours)-1, lambda x:x)

while True:
    target = img.copy() # 매 프레임마다 원본 복사
    index = cv.getTrackbarPos("Index", name) # 트랙바에서 현재 값 읽기
    x, y, width, height = cv.boundingRect(filtered_contours[index])
    cv.rectangle(target, (x,y), (x+width, y+height), (0,255,0), 2)

    cv.imshow(name, target)

    if cv.waitKey(1) == ord("q"):
        break

cv.destroyAllWindows()

In [20]:
# 실습5. 카드 하나씩 새 창에 표시
Playing_Cards_PATH = "../images/playing_cards.png"
img = cv.imread(Playing_Cards_PATH)
coppied = img.copy()
gray = cv.cvtColor(img, cv.COLOR_BGR2GRAY)
ret, binary = cv.threshold(gray, -1, 255, cv.THRESH_BINARY | cv.THRESH_OTSU)
contours, _ = cv.findContours(binary, cv.RETR_EXTERNAL, cv.CHAIN_APPROX_SIMPLE)

for idx, contour in enumerate(contours): # 윤곽선을 순회하며 카드 후보만 처리
    if cv.contourArea(contour) > 1300:
        x, y, width, height = cv.boundingRect(contour)

        crop = img[y:y+height, x:x+width] # 카드 영역 잘라내기
        cv.imshow(f"Card {idx}", crop) # 잘라낸 카드 이미지를 새 창으로 표시

        cv.rectangle(img, (x, y), (x + width, y + height), (0, 255, 0), 4)

cv.imshow("Bounding Rect", img)
cv.waitKey(0)
cv.destroyAllWindows()


In [ ]:
# 실습5. 카드 하나씩 새 창에 표시(트랙바 이용)
Playing_Cards_PATH = "../images/playing_cards.png"
img = cv.imread(Playing_Cards_PATH)
coppied = img.copy()
gray = cv.cvtColor(img, cv.COLOR_BGR2GRAY)
ret, binary = cv.threshold(gray, -1, 255, cv.THRESH_BINARY | cv.THRESH_OTSU)
contours, _ = cv.findContours(binary, cv.RETR_EXTERNAL, cv.CHAIN_APPROX_SIMPLE)

crops = [] # 잘라낸 카드 이미지 담을 리스트
for idx, contour in enumerate(contours):
    if cv.contourArea(contour) > 1300:
        x, y, width, height = cv.boundingRect(contour)
        cv.rectangle(img, (x, y), (x + width, y + height), (0, 255, 0), 4)
        crop = img[y:y+height, x:x+width]
        crops.append(crop)

# 트랙바 콜백 : 현재 인덱스(val)에 해당하는 카드 보여주기
def on_trackbar(val):
    target = crops[val] # 리스트에서 선택
    cv.imshow("Card", target) # 개별 카드 창에 표시

name = "Origianl"
cv.namedWindow(name)
# 트랙바 생성 : 범위(0 ~ len(crops)-1), 이동시 on_trackbar 호출 → "Card" 창에 업데이트
cv.createTrackbar("Index", name, 0, len(crops)-1, on_trackbar)

cv.imshow(name, img)
cv.waitKey(0)
cv.destroyAllWindows()
